# Library Book Recommendation System

## Data Description

In [1]:
#Importing the necessary libraries
import pandas as pd
import numpy as np

In [2]:
#Loading the datasets
rt_df = pd.read_csv("data/BX-Book-Ratings.csv", sep=";", error_bad_lines=False, encoding="latin-1")
bk_df = pd.read_csv("data/BX-Books.csv", sep=";", error_bad_lines=False, encoding="latin-1")
us_df = pd.read_csv("data/BX-Users.csv", sep=";", error_bad_lines=False, encoding="latin-1")

/var/folders/nn/vxj31dzx2ws_fw98wqh_r6wh0000gp/T/ipykernel_9317/784861674.py:2: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  rt_df = pd.read_csv("data/BX-Book-Ratings.csv", sep=";", error_bad_lines=False, encoding="latin-1")
/var/folders/nn/vxj31dzx2ws_fw98wqh_r6wh0000gp/T/ipykernel_9317/784861674.py:3: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  bk_df = pd.read_csv("data/BX-Books.csv", sep=";", error_bad_lines=False, encoding="latin-1")
b'Skipping line 6452: expected 8 fields, saw 9\nSkipping line 43667: expected 8 fields, saw 10\nSkipping line 51751: expected 8 fields, saw 9\n'
b'Skipping line 92038: expected 8 fields, saw 9\nSkipping line 104319: expected 8 fields, saw 9\nSkipping line 121768: expected 8 fields, saw 9\n'
b'Skipping line 144058: expected 8 fields, saw 9\nSkipping line 150789: e

In [3]:
# class for data understanding

class DataExplorer:
    
    def __init__(self, data):
        self.data = data

    def get_shape(self):
        return self.data.shape

    def get_info(self):
        return self.data.info()

    def get_missing_values(self):
        return self.data.isnull().sum()

    def get_duplicates(self):
        return self.data.duplicated().sum()

    def get_descriptive_stats(self):
        return self.data.describe()

In [4]:
explorer = DataExplorer(rt_df)

In [5]:
print(f'shape of the data {explorer.get_shape()}') 

shape of the data (1149780, 3)


In [6]:
print(f'\n A summary of the dataframe \n{explorer.get_info()}')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1149780 entries, 0 to 1149779
Data columns (total 3 columns):
 #   Column       Non-Null Count    Dtype 
---  ------       --------------    ----- 
 0   User-ID      1149780 non-null  int64 
 1   ISBN         1149780 non-null  object
 2   Book-Rating  1149780 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 26.3+ MB

 A summary of the dataframe 
None


In [7]:
print(f'Missing values are: \n{explorer.get_missing_values()}')

Missing values are: 
User-ID        0
ISBN           0
Book-Rating    0
dtype: int64


In [8]:
print(f'Number of duplicates {explorer.get_duplicates()}')

Number of duplicates 0


In [9]:
print(f'Descriptive statistics of the data:\n{explorer.get_descriptive_stats()}')

Descriptive statistics of the data:
            User-ID   Book-Rating
count  1.149780e+06  1.149780e+06
mean   1.403864e+05  2.866950e+00
std    8.056228e+04  3.854184e+00
min    2.000000e+00  0.000000e+00
25%    7.034500e+04  0.000000e+00
50%    1.410100e+05  0.000000e+00
75%    2.110280e+05  7.000000e+00
max    2.788540e+05  1.000000e+01


In [22]:
rt_df.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


**Ratings Data** has:
>* **3 columns** and **1,149,780** rows.
>* 2 datatypes : **int64** and **object**
>* No missing values
>* No duplicates

In [10]:
explorer = DataExplorer(bk_df)

In [11]:
print(f'shape of the data {explorer.get_shape()}') 

shape of the data (271360, 8)


In [12]:
print(f'\n A summary of the dataframe \n{explorer.get_info()}')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271360 entries, 0 to 271359
Data columns (total 8 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   ISBN                 271360 non-null  object
 1   Book-Title           271360 non-null  object
 2   Book-Author          271359 non-null  object
 3   Year-Of-Publication  271360 non-null  object
 4   Publisher            271358 non-null  object
 5   Image-URL-S          271360 non-null  object
 6   Image-URL-M          271360 non-null  object
 7   Image-URL-L          271357 non-null  object
dtypes: object(8)
memory usage: 16.6+ MB

 A summary of the dataframe 
None


In [13]:
print(f'Missing values are: \n{explorer.get_missing_values()}')

Missing values are: 
ISBN                   0
Book-Title             0
Book-Author            1
Year-Of-Publication    0
Publisher              2
Image-URL-S            0
Image-URL-M            0
Image-URL-L            3
dtype: int64


In [14]:
print(f'Number of duplicates {explorer.get_duplicates()}')

Number of duplicates 0


In [15]:
print(f'Descriptive statistics of the data:\n{explorer.get_descriptive_stats()}')

Descriptive statistics of the data:
              ISBN      Book-Title      Book-Author  Year-Of-Publication  \
count       271360          271360           271359               271360   
unique      271360          242135           102023                  202   
top     0195153448  Selected Poems  Agatha Christie                 2002   
freq             1              27              632                13903   

        Publisher                                        Image-URL-S  \
count      271358                                             271360   
unique      16807                                             271044   
top     Harlequin  http://images.amazon.com/images/P/185326119X.0...   
freq         7535                                                  2   

                                              Image-URL-M  \
count                                              271360   
unique                                             271044   
top     http://images.amazon.com/images

In [23]:
bk_df.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


**Books Data** has:
>*  **8 columns** and **271,359** rows.
>* 1 datatype: **object**
>* Missing values:
 >>* Book author: 1 
 >>* Publisher: 2
 >>* Image-URL-L: 3
>* No duplicates

In [16]:
explorer = DataExplorer(us_df)

In [17]:
print(f'shape of the data {explorer.get_shape()}') 

shape of the data (278858, 3)


In [18]:
print(f'\n A summary of the dataframe \n{explorer.get_info()}')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 278858 entries, 0 to 278857
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   User-ID   278858 non-null  int64  
 1   Location  278858 non-null  object 
 2   Age       168096 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 6.4+ MB

 A summary of the dataframe 
None


In [19]:
print(f'Missing values are: \n{explorer.get_missing_values()}')

Missing values are: 
User-ID          0
Location         0
Age         110762
dtype: int64


In [20]:
print(f'Number of duplicates {explorer.get_duplicates()}')

Number of duplicates 0


In [21]:
print(f'Descriptive statistics of the data:\n{explorer.get_descriptive_stats()}')

Descriptive statistics of the data:
            User-ID            Age
count  278858.00000  168096.000000
mean   139429.50000      34.751434
std     80499.51502      14.428097
min         1.00000       0.000000
25%     69715.25000      24.000000
50%    139429.50000      32.000000
75%    209143.75000      44.000000
max    278858.00000     244.000000


In [24]:
us_df.head()

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


**Users Data** has:
>* **3 columns** and **278,857** rows.
>* 3 datatypes **int64**, **float64** and **object**
>* Missing values:
 >>* Age: 110,762
>* No duplicates

## Data Cleaning

### Duplicates